Initialize Repos

In [ ]:
!pip install gdown --upgrade
!pip install overcomplete
!pip install einops

In [ ]:
import os
os.makedirs("oracle_saes", exist_ok=True)
os.makedirs("oracle_selection_models", exist_ok=True)
os.makedirs("./domainbed/data", exist_ok=True)

In [ ]:
!curl -L -o ./domainbed/data/PACS.zip https://www.kaggle.com/api/v1/datasets/download/ma3ple/pacs-dataset

In [ ]:
!unzip ./domainbed/data/PACS.zip -d ./domainbed/data/
!mv -f ./domainbed/data/kfold ./domainbed/data/PACS

### Model Trainer

In [ ]:
import os
import subprocess
import torch

# --- CONFIGURATION ---
data_dir = r"C:\Users\sproj_ha\Desktop\domainbed\domainbed\data"
dataset = "PACS"
algorithms = ["ERM", "IRM", "DANN", "CORAL", "Mixup", "MMD", "SagNet"]
test_env_combinations = ["0 1", "0 2", "0 3", "1 0", "1 2", "1 3", "2 3"]
backbones = ["ResNet50", "VIT", "DINO"]
output_root = "./results/PACS_Models"
base_cmd = "python -m domainbed.scripts.train"

# --- RUN ALL COMMANDS SEQUENTIALLY ---
for alg in algorithms:
    for test_envs in test_env_combinations:
        for backbone in backbones:
            hparams = {
                "ResNet50": "{\"resnet50_augmix\": true, \"vit\": false, \"dinov2\": false}",
                "VIT": "{\"resnet50_augmix\": false, \"vit\": true, \"dinov2\": false}",
                "DINO": "{\"resnet50_augmix\": false, \"vit\": true, \"dinov2\": true}"
            }[backbone]
    
            output_dir = f"{output_root}/{alg}_{backbone}_T{test_envs.replace(' ', '')}"
                        
            cmd = (
                f'{base_cmd} '
                f'--data_dir "{data_dir}" '
                f'--dataset {dataset} '
                f'--algorithm {alg} '
                f'--hparams "{hparams}" '
                f'--test_envs {test_envs} '
                f'--output_dir {output_dir} '
                f'--save_model_every_checkpoint '
                f'--hparams_seed 0 --trial_seed 0 --seed 0'
            )

            print(f"\n🚀 Running: {cmd}\n")
            try:
                subprocess.run(cmd, shell=True, check=True)
            except subprocess.CalledProcessError as e:
                print(f"❌ Command failed: {e}")
            
            # --- CLEAR CACHE BETWEEN RUNS ---
            try:
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    print("🧹 Cleared CUDA cache.")
            except Exception as e:
                print(f"⚠️ Could not clear CUDA cache: {e}")
            
            # Optionally clear OS-level file system cache (Linux only)
            if os.name == "posix":
                os.system("sync; echo 3 | sudo tee /proc/sys/vm/drop_caches > /dev/null 2>&1")
                print("🧽 Dropped system caches (Linux).")

print("\n✅ All commands finished running sequentially.")


### Run Oracle Selection on Trained Models

In [ ]:
import re
import pandas as pd
import shutil

# === Helper functions ===

def parse_test_envs(folder_name, file_text):
    """Extract test environment indices from folder name (e.g. '_T12' -> [1, 2])"""
    match = re.search(r"_T(\d+)", folder_name)
    if match:
        digits = list(match.group(1))
        return [int(d) for d in digits]
    # Fallback if missing in name: try reading from out.txt header
    m2 = re.search(r"test_envs:\s*\[([0-9,\s]+)\]", file_text)
    if m2:
        return [int(x.strip()) for x in m2.group(1).split(",")]
    return []

def parse_table(file_text):
    """Parse the main results table from out.txt, robust to DANN spacing and header variants."""
    # Find all candidate headers
    matches = list(re.finditer(r"^(disc_loss|env0_in_acc)\s+", file_text, re.MULTILINE))
    if not matches:
        raise ValueError("Could not locate results table start (env0_in_acc or disc_loss)")

    # Use the last one (main training section)
    start_idx = matches[-1].start()
    table_text = file_text[start_idx:].strip()
    lines = [ln.strip() for ln in table_text.splitlines() if ln.strip()]

    # Handle case where the first header is env0_in_acc but next is disc_loss
    if lines[0].startswith("env0_in_acc") and len(lines) > 1 and lines[1].startswith("disc_loss"):
        lines = lines[1:]

    header = re.split(r"\s{2,}|\t+", lines[0].strip())
    data_lines = []
    for line in lines[1:]:
        if line.startswith("Updated state") or line.startswith("Restarting"):
            break
        parts = re.split(r"\s{2,}|\t+", line.strip())
        if len(parts) == len(header):
            data_lines.append(parts)

    if not data_lines:
        raise ValueError("No valid rows found in results table")

    df = pd.DataFrame(data_lines, columns=header)
    # Convert numeric columns
    df = df.apply(pd.to_numeric, errors="ignore")
    return df

def get_best_checkpoint(root_dir):
    """Traverse experiment folders, read out.txt, and find best model per test env."""
    results = []

    for folder in sorted(os.listdir(root_dir)):
        folder_path = os.path.join(root_dir, folder)
        out_file = os.path.join(folder_path, "out.txt")
        if not os.path.isfile(out_file):
            continue

        try:
            with open(out_file, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()
        except Exception as e:
            print(f"⚠️ Failed to read {folder}: {e}")
            continue

        try:
            df = parse_table(text)
        except Exception as e:
            print(f"⚠️ Failed to parse table in {folder}: {e}")
            continue

        test_envs = parse_test_envs(folder, text)
        if not test_envs:
            print(f"⚠️ No test envs found in {folder}")
            continue

        env_accs = [f"env{t}_out_acc" for t in test_envs if f"env{t}_out_acc" in df.columns]
        if not env_accs:
            print(f"⚠️ No env*_out_acc columns found in {folder}")
            continue

        # Ensure numeric
        df[env_accs] = df[env_accs].apply(pd.to_numeric, errors="coerce")
        df["weighted_acc"] = df[env_accs].mean(axis=1)

        best_idx = df["weighted_acc"].idxmax()
        best_row = df.loc[best_idx]

        best_step = str(int(best_row["step"])) if "step" in best_row else "N/A"

        results.append({
            "folder": folder,
            "folder_path": folder_path,
            "best_checkpoint": best_step,
            "weighted_acc": best_row["weighted_acc"],
            **{env_acc: best_row[env_acc] for env_acc in env_accs},
        })

    return results


def copy_best_checkpoints(results, root_dir):
    """Copy best checkpoint files into Oracle Selection folder."""
    oracle_dir = os.path.join(root_dir, "Oracle Selection")
    os.makedirs(oracle_dir, exist_ok=True)

    for r in results:
        src_folder = r["folder_path"]
        dst_folder = os.path.join(oracle_dir, r["folder"])
        os.makedirs(dst_folder, exist_ok=True)

        step = r["best_checkpoint"]
        if step == "N/A":
            print(f"⚠️ Skipping {r['folder']} (no valid checkpoint step)")
            continue

        checkpoint_name = f"model_step{step}.pkl"
        src = os.path.join(src_folder, checkpoint_name)
        dst = os.path.join(dst_folder, checkpoint_name)

        if os.path.exists(src):
            shutil.copy2(src, dst)
            print(f"✅ Copied {checkpoint_name} → {dst_folder}")
        else:
            print(f"⚠️ No checkpoint found for step {step} in {r['folder']} (expected {checkpoint_name})")

In [ ]:
root_dir = r""  # 🔁 change this to your experiments root
results = get_best_checkpoint(root_dir)

print("\n=== Oracle Model Selection Results ===\n")
for r in results:
    print(f"📁 {r['folder']}")
    print(f"  → Best checkpoint: {r['best_checkpoint']}")
    print(f"  → Weighted Acc: {r['weighted_acc']:.4f}")
    for k, v in r.items():
        if k.startswith("env") and k.endswith("_out_acc"):
            print(f"     {k}: {v:.4f}")
    print()

print("📦 Copying best checkpoints to 'Oracle Selection'...\n")
copy_best_checkpoints(results, root_dir)
print("\n✅ Done! All best checkpoints collected under 'Oracle Selection'.")


In [ ]:
import shutil
shutil.rmtree(root_dir, ignore_errors=True)
print(f"Deleted: {root_dir}")

### SAE Trainer

In [ ]:
from saetrainer_shell import run
run()